## Coleta de dados de recursos disponíveis para Os Objetivos de Desenvolvimento Sustentável (ODS)

Preparação dos dados para consumo no dashboard streamlit

### Fonte de dados
- url : https://brasil.un.org/pt-br/sdgs

### Atualizações
- 27/02/2024 : Início do projeto

### Importação das dependências

In [47]:
import pandas as pd
import plotly.express as px
from selenium.webdriver import ActionChains, Keys
from selenium.webdriver.common.by import By
from pathlib import Path

from config import Config
from open_data_day_2024.web_page import get_element_by_xpath, get_page_driver

### Localização do arquivos

In [48]:
summary_file = Path.cwd() / "data" / "processed" / f"summary_recursos_disponiveis.pkl"

### Seletores XPATH

In [2]:
COMBOBOX_ANO_XPATH = ('//h2[contains(text(), "Onde são investidos os recursos?")]'
                      '//ancestor::div[3]//label[@id="Year"]//following::div[1]')
RECURSOS_DISPONIVEIS_XPATH = '//h4[contains(text(),"Recursos Disponíveis")]'
ITEM_ODS_DIST_RECURSOS_XPATH = '//ul[contains(@class,"list")]//strong//parent::div'

### URL da página

In [41]:
Config.URL

'https://brasil.un.org/pt-br/sdgs'

### Cria uma instância do ChromeWebDriver e abre a página

In [42]:
driver = get_page_driver(Config.URL)

### Seleciona o elemento selectbox na DOM e move até a posição dele na página
Document Object Model (DOM): No contexto da programação web, DOM refere-se ao Document Object Model, que é uma interface de programação para documentos HTML, XML e SVG. Ele representa a estrutura do documento como uma árvore de objetos, onde cada nó representa parte do documento, como elementos HTML, atributos e texto.

In [5]:
selectbox_ano = get_element_by_xpath(driver, COMBOBOX_ANO_XPATH)
ActionChains(driver).move_to_element(selectbox_ano).perform()
selectbox_ano.text

'2024'

### Lista todo os anos encontrados na selectbox

In [6]:
selectbox_ano = get_element_by_xpath(driver, COMBOBOX_ANO_XPATH)
ActionChains(driver).click(selectbox_ano).perform()
lista_anos_selectbox = driver.find_elements(By.XPATH, "//li[contains(text(), '20')]")
ANOS_SELECTBOX = list(set([ano.text for ano in lista_anos_selectbox if ano.text]))
lista_anos_selectbox[-1].send_keys(Keys.ESCAPE)
print(ANOS_SELECTBOX)

['2027', '2024', '2025', '2026', '2023']


### Altera o ano no elemento selectbox e imprime o ano selecionado

In [7]:
import time
ANO = 2023

while True:
    selectbox_ano = get_element_by_xpath(driver, COMBOBOX_ANO_XPATH)
    if selectbox_ano.text == str(ANO):
        break
    ActionChains(driver).click(selectbox_ano).perform()
    
    item_to_select = get_element_by_xpath(driver, f"//li[contains(text(), '{ANO}')]")
    ActionChains(driver).click(item_to_select).perform()
      
    selectbox_ano = get_element_by_xpath(driver, COMBOBOX_ANO_XPATH)
    if selectbox_ano.text == str(ANO):
        break
    
selectbox_ano = get_element_by_xpath(driver, COMBOBOX_ANO_XPATH)
ActionChains(driver).move_to_element(selectbox_ano).perform()
selectbox_ano.text

'2024'

### Seleciona o elemento "recursos diponíveis" e exibe o seu conteúdo

In [8]:
recursos_disponiveis = get_element_by_xpath(driver, RECURSOS_DISPONIVEIS_XPATH)
recursos_disponiveis.text

'Recursos Disponíveis $110,1 million'

### Seleciona a lista de itens do OSD com suas respectivas informações

In [23]:
item_ods_recurso_list = driver.find_elements(By.XPATH, ITEM_ODS_DIST_RECURSOS_XPATH)
distribuicao_recursos = list()
for item in item_ods_recurso_list:
    titulo, percentual = item.text.splitlines()
    percentual = float(percentual.split('%')[0].replace(',', '.'))
    distribuicao_recursos.append([titulo, percentual])
    print(f'{titulo:.<50}', percentual)



1. Erradicação da pobreza......................... 3.6
2. Fome zero e agricultura sustentável............ 8.3
3. Saúde e Bem-Estar.............................. 11.3
4. Educação de qualidade.......................... 6.5
5. Igualdade de género............................ 2.5
6. Água potável e saneamento...................... 3.5
7. Energia limpa e acessível...................... 0.0
8. Trabalho decente e crescimento económico....... 9.4
9. Indústria, inovação e infraestrutura........... 2.7
10. Redução das desigualdades..................... 2.2
11. Cidades e comunidades sustentáveis............ 8.4
12. Consumo e produção responsáveis............... 1.7
13. Ação contra a mudança global do clima......... 2.6
14. Vida na água.................................. 0.1
15. Vida terrestre................................ 5.1
16. Paz, justiça e instituições eficazes.......... 16.0
17. Parcerias e meios de implementação............ 16.2


### Fecha o navegador

In [ ]:
driver.close()

### Função que centraliza todas as ações realizada acima

In [34]:
def get_dados_ods(ano: int):
    """Raspa os dados de recursos disponíveis para Os ODS com base no ano

    Args:
        ano (int): ano dos recursos disponíveis

    Returns:
        Dict[str, Any]: retorna os dados raspados inclusos em um dicionário
    """
    
    driver = get_page_driver(Config.URL)

    while True:
        selectbox_ano = get_element_by_xpath(driver, COMBOBOX_ANO_XPATH)
        ActionChains(driver).move_to_element(selectbox_ano).perform()
        if selectbox_ano.text == str(ano):
            break
        ActionChains(driver).click(selectbox_ano).perform()
        
        item_to_select = get_element_by_xpath(driver, f"//li[contains(text(), {ano})]")
        ActionChains(driver).click(item_to_select).perform()
        
        selectbox_ano = get_element_by_xpath(driver, COMBOBOX_ANO_XPATH)
        if selectbox_ano.text == str(ano):
            break   

    recursos_disponiveis = get_element_by_xpath(driver, RECURSOS_DISPONIVEIS_XPATH)
    recursos_disponiveis = recursos_disponiveis.text
    recursos_valores, recurso_notacao = recursos_disponiveis.split('$')[-1].split()
    recursos_valores = recursos_valores.replace(',', '.')

    if 'billion' in recurso_notacao:
        recursos_valores = float(recursos_valores) * 1_000_000_000
    elif 'million' in recurso_notacao:
        recursos_valores = float(recursos_valores) * 1_000_000
    elif 'thousand' in recurso_notacao:
        recursos_valores = float(recursos_valores) * 1_000

    dados_dist_recursos = {
        'ano': ano,
        'recursos_disponiveis': recursos_disponiveis,
        'recursos_valores': recursos_valores,
    }

    if recursos_disponiveis is not None:
        item_ods_recurso_list = driver.find_elements(By.XPATH, ITEM_ODS_DIST_RECURSOS_XPATH)
        distribuicao_recursos = list()        
        for item in item_ods_recurso_list:
            titulo, percentual = item.text.splitlines()
            percentual = float(percentual.split('%')[0].replace(',', '.'))
            distribuicao_recursos.append([titulo, percentual])
        dados_dist_recursos.update({'distribuicao_recursos': distribuicao_recursos})

    return dados_dist_recursos

get_dados_ods(2024)

{'ano': 2024,
 'recursos_disponiveis': 'Recursos Disponíveis $110,1 million',
 'recursos_valores': 110100000.0,
 'distribuicao_recursos': [['1. Erradicação da pobreza', 3.6],
  ['2. Fome zero e agricultura sustentável', 8.3],
  ['3. Saúde e Bem-Estar', 11.3],
  ['4. Educação de qualidade', 6.5],
  ['5. Igualdade de género', 2.5],
  ['6. Água potável e saneamento', 3.5],
  ['7. Energia limpa e acessível', 0.0],
  ['8. Trabalho decente e crescimento económico', 9.4],
  ['9. Indústria, inovação e infraestrutura', 2.7],
  ['10. Redução das desigualdades', 2.2],
  ['11. Cidades e comunidades sustentáveis', 8.4],
  ['12. Consumo e produção responsáveis', 1.7],
  ['13. Ação contra a mudança global do clima', 2.6],
  ['14. Vida na água', 0.1],
  ['15. Vida terrestre', 5.1],
  ['16. Paz, justiça e instituições eficazes', 16.0],
  ['17. Parcerias e meios de implementação', 16.2]]}

### Converte os dados em DataFrames pandas

In [35]:
df_dict = {}
for ano in sorted(ANOS_SELECTBOX):
    print(ano)

    # busca os dados que cada ano constante na selectbox ano dos recursos disponíveis
    dados = get_dados_ods(ano)
    
    # converte em DataFrames os dados coletados
    df = pd.DataFrame(dados['distribuicao_recursos'], columns=['Objetivo', 'Percentual'])
    df['Ano'] = dados['ano']
    df['Valor'] = df['Percentual']
    df['Valor'] = df['Valor'].apply(lambda x: (x / 100) * dados['recursos_valores'])

    # adiciona os DataFrames e seus respectivos dados no dicionário
    df_dict[ano] = (dados, df)

2023
2024
2025
2026
2027


### Itera sobre o dicionário e exibe o conteúdo dos Dataframes

In [36]:
for ano, registro in df_dict.items():
    dados, df = registro
    print(ano, '-', dados['recursos_disponiveis'])
    display(df)

2023 - Recursos Disponíveis $187 million


,Objetivo,Percentual,Ano,Valor
0,1. Erradicação da pobreza,13.6,2023,25432000.0
1,2. Fome zero e agricultura sustentável,7.5,2023,14025000.0
2,3. Saúde e Bem-Estar,5.5,2023,10285000.0
3,4. Educação de qualidade,11.0,2023,20570000.0
4,5. Igualdade de género,5.0,2023,9350000.0
5,6. Água potável e saneamento,2.2,2023,4114000.0
6,7. Energia limpa e acessível,0.0,2023,0.0
7,8. Trabalho decente e crescimento económico,6.3,2023,11781000.0
8,"9. Indústria, inovação e infraestrutura",3.4,2023,6358000.0
9,10. Redução das desigualdades,6.7,2023,12529000.0


2024 - Recursos Disponíveis $110,1 million


,Objetivo,Percentual,Ano,Valor
0,1. Erradicação da pobreza,3.6,2024,3963600.0
1,2. Fome zero e agricultura sustentável,8.3,2024,9138300.0
2,3. Saúde e Bem-Estar,11.3,2024,12441300.0
3,4. Educação de qualidade,6.5,2024,7156500.0
4,5. Igualdade de género,2.5,2024,2752500.0
5,6. Água potável e saneamento,3.5,2024,3853500.0
6,7. Energia limpa e acessível,0.0,2024,0.0
7,8. Trabalho decente e crescimento económico,9.4,2024,10349400.0
8,"9. Indústria, inovação e infraestrutura",2.7,2024,2972700.0
9,10. Redução das desigualdades,2.2,2024,2422200.0


2025 - Recursos Disponíveis $6,2 million


,Objetivo,Percentual,Ano,Valor
0,1. Erradicação da pobreza,14.4,2025,892800.0
1,2. Fome zero e agricultura sustentável,0.0,2025,0.0
2,3. Saúde e Bem-Estar,1.8,2025,111600.0
3,4. Educação de qualidade,0.0,2025,0.0
4,5. Igualdade de género,0.0,2025,0.0
5,6. Água potável e saneamento,0.0,2025,0.0
6,7. Energia limpa e acessível,0.0,2025,0.0
7,8. Trabalho decente e crescimento económico,48.7,2025,3019400.0
8,"9. Indústria, inovação e infraestrutura",10.2,2025,632400.0
9,10. Redução das desigualdades,16.2,2025,1004400.0


2026 - Recursos Disponíveis $4,8 million


,Objetivo,Percentual,Ano,Valor
0,1. Erradicação da pobreza,18.7,2026,897600.0
1,2. Fome zero e agricultura sustentável,0.0,2026,0.0
2,3. Saúde e Bem-Estar,0.0,2026,0.0
3,4. Educação de qualidade,0.0,2026,0.0
4,5. Igualdade de género,0.0,2026,0.0
5,6. Água potável e saneamento,0.0,2026,0.0
6,7. Energia limpa e acessível,0.0,2026,0.0
7,8. Trabalho decente e crescimento económico,62.5,2026,3000000.0
8,"9. Indústria, inovação e infraestrutura",0.0,2026,0.0
9,10. Redução das desigualdades,18.7,2026,897600.0


2027 - Recursos Disponíveis $500 thousand


,Objetivo,Percentual,Ano,Valor
0,1. Erradicação da pobreza,0.0,2027,0.0
1,2. Fome zero e agricultura sustentável,0.0,2027,0.0
2,3. Saúde e Bem-Estar,0.0,2027,0.0
3,4. Educação de qualidade,0.0,2027,0.0
4,5. Igualdade de género,0.0,2027,0.0
5,6. Água potável e saneamento,0.0,2027,0.0
6,7. Energia limpa e acessível,0.0,2027,0.0
7,8. Trabalho decente e crescimento económico,100.0,2027,500000.0
8,"9. Indústria, inovação e infraestrutura",0.0,2027,0.0
9,10. Redução das desigualdades,0.0,2027,0.0


# Grafico de pizza - Recursos Disponíveis 2024

Dentro da classificação de gráficos, o gráfico de pizza (ou pie chart, em inglês) é geralmente categorizado como um tipo de gráfico de distribuição.

Os gráficos de distribuição são usados para representar a distribuição de dados, mostrando como os diferentes valores ou categorias contribuem para um todo. No caso do gráfico de pizza, ele mostra a proporção de cada categoria em relação ao todo, dividindo um círculo em segmentos proporcionais aos valores que representam.

In [43]:
dados, df = df_dict['2024']
fig = px.pie(df, values='Percentual', names='Objetivo', 
             title=f'Distribuição de ${dados["recursos_valores"]:,.2f} entre os ODS {dados["ano"]}')
fig.show()

In [38]:
import plotly.graph_objects as go

# pull is given as a fraction of the pie radius
fig = go.Figure(data=[go.Pie(labels=df['Objetivo'], 
                             values=df['Percentual'], pull= 16*[0]+[0.2])])
fig.show()

### Manipulação de dados

In [44]:
df = pd.concat([v[1] for k, v in df_dict.items()])
df

,Objetivo,Percentual,Ano,Valor
0,1. Erradicação da pobreza,13.6,2023,25432000.0
1,2. Fome zero e agricultura sustentável,7.5,2023,14025000.0
2,3. Saúde e Bem-Estar,5.5,2023,10285000.0
3,4. Educação de qualidade,11.0,2023,20570000.0
4,5. Igualdade de género,5.0,2023,9350000.0
...,...,...,...,...
12,13. Ação contra a mudança global do clima,0.0,2027,0.0
13,14. Vida na água,0.0,2027,0.0
14,15. Vida terrestre,0.0,2027,0.0
15,"16. Paz, justiça e instituições eficazes",0.0,2027,0.0


### Salve o arquivo de saida dentro do diretório processed

Save a file in the processed directory that is cleaned properly. It will be read in and used later for further analysis.
Salve o arquivo no diretório processed que esta propriamente limpo. Ele será lido e usado mais tarde para outras análises.
Outras opções além do pickle incluem:
- feather: é um formato de arquivo binário para armazenamento eficiente de dados em data frames. É projetado para   interoperabilidade entre linguagens de programação, sendo especialmente eficaz para dados tabulares. Feather é rápido para leitura e gravação, e é compatível com R e Python, tornando-o útil para transferir dados entre essas duas 
- msgpack: é um formato de serialização de dados binários que é mais eficiente em termos de espaço e tempo de processamento do que o formato JSON. Ele é projetado para ser rápido e eficiente em termos de espaço de armazenamento, e pode ser utilizado para serializar uma ampla variedade de tipos de dados, tornando-o útil para comunicação de dados entre sistemas distribuídos.
- parquet: é um formato de arquivo colunar projetado para armazenar dados tabulares de forma eficiente em termos de espaço e tempo de acesso. Ele é especialmente útil para armazenar e analisar grandes conjuntos de dados, pois permite a leitura eficiente de subconjuntos de colunas, o que pode acelerar a análise de consultas. Parquet é comumente usado em ambientes de big data, como Hadoop e Apache Spark.

In [53]:
df.to_pickle(summary_file)